In [1]:
import numpy as np
import torch

# visualization
import matplotlib as mpl
import matplotlib.pyplot as plt

# sbi
from sbi import utils as utils
from sbi import analysis as analysis
from sbi.inference import SNLE, prepare_for_sbi, simulate_for_sbi
from sbi.inference.base import infer

/Users/ryankelly/python_projects/sbi/venv/lib/python3.9/site-packages/tqdm-4.64.0-py3.9.egg/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# remove top and right axis from plots
mpl.rcParams['axes.spines.right'] = False
mpl.rcParams['axes.spines.top'] = False

In [3]:
def AR1(phi, n_obs=200, batch_size=1, random_state=None):
    r"""Generate a sequence of samples from the AR1 model.

    The sequence is an autoregressive model

        x_i = \phi x_{i-1} + w_{i}

    where w_i are white noise ~ N(0,1) and x_0 = 0.

    Parameters
    ----------
    phi : float, array_like
    n_obs : int, optional
    batch_size : int, optional
    random_state : RandomState, optional

    """
    phi = np.asanyarray(phi)
    random_state = random_state or np.random

    # i.i.d. sequence ~ N(0,1)
    w = random_state.randn(batch_size, n_obs + 1)
    x = np.zeros((batch_size, n_obs+1))
    x_prev = np.zeros(batch_size)
    for i in range(1, n_obs+1):
        x[:, i] = phi * x_prev + w[:, i]
        x_prev = x[:, i]
    return x[:, 1:]


In [4]:
prior_min = [-1]
prior_max = [1]
prior = utils.torchutils.BoxUniform(low=torch.as_tensor(prior_min),
                                    high=torch.as_tensor(prior_max))

In [5]:
def simulation_wrapper(params):
    x_sim = AR1(params)

    # sim_sum = torch.as_tensor(summstats(x_sim).astype('float32'))
    sim_sum = torch.as_tensor(x_sim.astype('float32'))
    return sim_sum.reshape((-1, 200))  # TODO: magic number 100

In [6]:
inference = SNLE(prior=prior)

In [8]:
true_params = 0.9
y = simulation_wrapper(true_params)

num_rounds = 10

posteriors = []
proposal = prior

for _ in range(num_rounds):
    theta, x = simulate_for_sbi(simulation_wrapper, proposal, num_simulations=500)
    density_estimator = inference.append_simulations(theta, x
    ).train()
    posterior = inference.build_posterior(density_estimator)
    posteriors.append(posterior)
    proposal = posterior.set_default_x(y)


Running 500 simulations.: 100%|██████████| 500/500 [00:01<00:00, 279.96it/s]


 Neural network successfully converged after 80 epochs.

Running 500 simulations.: 100%|██████████| 500/500 [00:01<00:00, 397.43it/s]


 Training neural network. Epochs trained: 236

KeyboardInterrupt: 

In [ ]:
samples = posterior.sample((10000,),
                           x=y)

In [ ]:
fig, axes = analysis.pairplot(samples,
                           limits=[[-1, 1], [-1, 1]],
                        #    ticks=[[.5, 1], [.5, 15.]],
                           figsize=(5,5),
                        #    points=true_params,
                           points_offdiag={'markersize': 6},
                           points_colors='r');